## Step 1: link to the API

In [121]:
import pandas as pd
from sqlalchemy import create_engine
import json
from pandas.io.json import json_normalize
import requests
from config import key
from config import gkey
import time
import pymongo


In [87]:
key = "greywolf16"

#get state code number --CA 06
state_url = "https://aqs.epa.gov/data/api/list/states?email="+gkey+"&key="+key
response = requests.get(state_url)
response_json = response.json()
response_json_data = response_json['Data']
#print(response_json_data)
state_df = pd.DataFrame.from_dict(response_json_data)
state_df.head()

,code,value_represented
0,01,Alabama
1,02,Alaska
2,04,Arizona
3,05,Arkansas
4,06,California


In [85]:
#get SF code -- 075
city_url = "https://aqs.epa.gov/data/api/list/countiesByState?email="+gkey +"&key="+key+"&state=06"
print(city_url)
response = requests.get(city_url)
response_json = response.json()
response_json_data = response_json['Data']
#print(response_json_data)
city_df = pd.DataFrame.from_dict(response_json_data)
city_df.head()

https://aqs.epa.gov/data/api/list/countiesByState?email=janecc.gan@gmail.com&key=greywolf16&state=06


,code,value_represented
0,001,Alameda
1,003,Alpine
2,005,Amador
3,007,Butte
4,009,Calaveras


In [10]:
#CBSAs code": "41860",
#      "value_represented": "San Francisco-Oakland-Hayward, CA

#  "Parameter code": "PM2.5 CONT NONREF",
#       "value_represented": "PM2.5 Continuous, Nonreference Methods"
#     },
#     {
#       "code": "PM2.5 MASS/QA",
#       "value_represented": "PM2.5 Mass and QA Parameters"


# "CRITERIA code": "88101",
#       "value_represented": "PM2.5 - Local Conditions"

In [49]:
#get current 5y PM 2.5 data for SF
#https://aqs.epa.gov/data/api/sampleData/byCBSA?email=test@aqs.api&key=test&param=42602&bdate=20170101&edate=20170101&cbsa=16740
#sf_url = "https://aqs.epa.gov/data/api/sampleData/byCBSA?email="+gkey+"&key="+key+"&param=88101&bdate=20190101&edate=20190101&cbsa=41860"

sf_url = "https://aqs.epa.gov/data/api/dailyData/byCounty?email="+gkey+"&key="+key+"&param=88101&bdate=20140301&edate=20190331&state=06&county=075"
print(sf_url)
response = requests.get(sf_url)
response_json = response.json()
#print(response_json)
response_json_data = response_json['Data']
# #print(response_json_data)
sf_df = pd.DataFrame.from_dict(response_json_data)
sf_df.head()

#https://aqs.epa.gov/data/api/monitors/byState?email=test@aqs.api&key=test&param=42401&bdate=20150501&edate=20150502&state=15

https://aqs.epa.gov/data/api/dailyData/byCounty?email=janecc.gan@gmail.com&key=greywolf16&param=88101&bdate=20140301&edate=20190331&state=06&county=075


,aqi,arithmetic_mean,cbsa,cbsa_code,city,county,county_code,date_local,date_of_last_change,datum,...,parameter_code,poc,pollutant_standard,sample_duration,site_address,site_number,state,state_code,units_of_measure,validity_indicator
0,28.0,6.7,"San Francisco-Oakland-Hayward, CA",41860,San Francisco,San Francisco,075,2019-03-31,2019-08-27,WGS84,...,88101,3,PM25 24-hour 2006,24-HR BLK AVG,10 ARKANSAS ST.,0005,California,06,Micrograms/cubic meter (LC),Y
1,22.0,5.2,"San Francisco-Oakland-Hayward, CA",41860,San Francisco,San Francisco,075,2019-03-30,2019-08-27,WGS84,...,88101,3,PM25 24-hour 2006,24-HR BLK AVG,10 ARKANSAS ST.,0005,California,06,Micrograms/cubic meter (LC),Y
2,23.0,5.4,"San Francisco-Oakland-Hayward, CA",41860,San Francisco,San Francisco,075,2019-03-29,2019-08-27,WGS84,...,88101,3,PM25 24-hour 2006,24-HR BLK AVG,10 ARKANSAS ST.,0005,California,06,Micrograms/cubic meter (LC),Y
3,24.0,5.7,"San Francisco-Oakland-Hayward, CA",41860,San Francisco,San Francisco,075,2019-03-28,2019-08-27,WGS84,...,88101,3,PM25 24-hour 2006,24-HR BLK AVG,10 ARKANSAS ST.,0005,California,06,Micrograms/cubic meter (LC),Y
4,15.0,3.5,"San Francisco-Oakland-Hayward, CA",41860,San Francisco,San Francisco,075,2019-03-27,2019-08-27,WGS84,...,88101,3,PM25 24-hour 2006,24-HR BLK AVG,10 ARKANSAS ST.,0005,California,06,Micrograms/cubic meter (LC),Y


In [61]:
# get 5year California data by county
county_codes = city_df['code'].tolist()
county_codes_10 = county_codes[0:10]
county_codes_10

['001', '003', '005', '007', '009', '011', '013', '015', '017', '019']

In [90]:
total_county_data_10 = pd.DataFrame()
for i in county_codes_10:
    county_data = "https://aqs.epa.gov/data/api/dailyData/byCounty?email="+gkey+"&key="+key+"&param=88101&bdate=20140301&edate=20190331&state=06&county="+i
    print(county_data)
    response = requests.get(county_data)
    response_json = response.json()
    response_json_data = response_json['Data']
    county_data =pd.DataFrame.from_dict(response_json_data)
    total_county_data_10 = total_county_data_10.append(county_data)
    time.sleep(2)
total_county_data_10.head()

https://aqs.epa.gov/data/api/dailyData/byCounty?email=janecc.gan@gmail.com&key=greywolf16&param=88101&bdate=20140301&edate=20190331&state=06&county=001
https://aqs.epa.gov/data/api/dailyData/byCounty?email=janecc.gan@gmail.com&key=greywolf16&param=88101&bdate=20140301&edate=20190331&state=06&county=003
https://aqs.epa.gov/data/api/dailyData/byCounty?email=janecc.gan@gmail.com&key=greywolf16&param=88101&bdate=20140301&edate=20190331&state=06&county=005
https://aqs.epa.gov/data/api/dailyData/byCounty?email=janecc.gan@gmail.com&key=greywolf16&param=88101&bdate=20140301&edate=20190331&state=06&county=007
https://aqs.epa.gov/data/api/dailyData/byCounty?email=janecc.gan@gmail.com&key=greywolf16&param=88101&bdate=20140301&edate=20190331&state=06&county=009
https://aqs.epa.gov/data/api/dailyData/byCounty?email=janecc.gan@gmail.com&key=greywolf16&param=88101&bdate=20140301&edate=20190331&state=06&county=011
https://aqs.epa.gov/data/api/dailyData/byCounty?email=janecc.gan@gmail.com&key=greywolf1

,aqi,arithmetic_mean,cbsa,cbsa_code,city,county,county_code,date_local,date_of_last_change,datum,...,parameter_code,poc,pollutant_standard,sample_duration,site_address,site_number,state,state_code,units_of_measure,validity_indicator
0,18.0,4.2,"San Francisco-Oakland-Hayward, CA",41860,Livermore,Alameda,001,2019-03-31,2019-08-27,WGS84,...,88101,3,PM25 24-hour 2006,24-HR BLK AVG,793 Rincon Ave.,0007,California,06,Micrograms/cubic meter (LC),Y
1,16.0,3.9,"San Francisco-Oakland-Hayward, CA",41860,Livermore,Alameda,001,2019-03-30,2019-08-27,WGS84,...,88101,3,PM25 24-hour 2006,24-HR BLK AVG,793 Rincon Ave.,0007,California,06,Micrograms/cubic meter (LC),Y
2,8.0,1.8,"San Francisco-Oakland-Hayward, CA",41860,Livermore,Alameda,001,2019-03-29,2019-08-27,WGS84,...,88101,3,PM25 24-hour 2006,24-HR BLK AVG,793 Rincon Ave.,0007,California,06,Micrograms/cubic meter (LC),Y
3,19.0,4.5,"San Francisco-Oakland-Hayward, CA",41860,Livermore,Alameda,001,2019-03-28,2019-08-27,WGS84,...,88101,3,PM25 24-hour 2006,24-HR BLK AVG,793 Rincon Ave.,0007,California,06,Micrograms/cubic meter (LC),Y
4,9.0,2.1,"San Francisco-Oakland-Hayward, CA",41860,Livermore,Alameda,001,2019-03-27,2019-08-27,WGS84,...,88101,3,PM25 24-hour 2006,24-HR BLK AVG,793 Rincon Ave.,0007,California,06,Micrograms/cubic meter (LC),Y


In [91]:
total_county_data_10.to_csv("ca_10.csv")

In [71]:
county_codes = city_df['code'].tolist()
county_codes_40 = county_codes[31:40]
county_codes_40

['063', '065', '067', '069', '071', '073', '075', '077', '079']

In [92]:
total_county_data_40 = pd.DataFrame()
for i in county_codes_40:
    county_data = "https://aqs.epa.gov/data/api/dailyData/byCounty?email="+gkey+"&key="+key+"&param=88101&bdate=20140301&edate=20190331&state=06&county="+i
    print(county_data)
    response = requests.get(county_data)
    response_json = response.json()
    response_json_data = response_json['Data']
    county_data =pd.DataFrame.from_dict(response_json_data)
    total_county_data_40 = total_county_data_40.append(county_data)
    time.sleep(2)
total_county_data_40.head()

https://aqs.epa.gov/data/api/dailyData/byCounty?email=janecc.gan@gmail.com&key=greywolf16&param=88101&bdate=20140301&edate=20190331&state=06&county=063
https://aqs.epa.gov/data/api/dailyData/byCounty?email=janecc.gan@gmail.com&key=greywolf16&param=88101&bdate=20140301&edate=20190331&state=06&county=065
https://aqs.epa.gov/data/api/dailyData/byCounty?email=janecc.gan@gmail.com&key=greywolf16&param=88101&bdate=20140301&edate=20190331&state=06&county=067
https://aqs.epa.gov/data/api/dailyData/byCounty?email=janecc.gan@gmail.com&key=greywolf16&param=88101&bdate=20140301&edate=20190331&state=06&county=069
https://aqs.epa.gov/data/api/dailyData/byCounty?email=janecc.gan@gmail.com&key=greywolf16&param=88101&bdate=20140301&edate=20190331&state=06&county=071
https://aqs.epa.gov/data/api/dailyData/byCounty?email=janecc.gan@gmail.com&key=greywolf16&param=88101&bdate=20140301&edate=20190331&state=06&county=073
https://aqs.epa.gov/data/api/dailyData/byCounty?email=janecc.gan@gmail.com&key=greywolf1

,aqi,arithmetic_mean,cbsa,cbsa_code,city,county,county_code,date_local,date_of_last_change,datum,...,parameter_code,poc,pollutant_standard,sample_duration,site_address,site_number,state,state_code,units_of_measure,validity_indicator
0,26.0,6.2,None,None,Quincy,Plumas,063,2019-03-31,2019-08-05,NAD83,...,88101,1,PM25 24-hour 2006,24 HOUR,"267 N. CHURCH ST., QUINCY, CA. 95971",1006,California,06,Micrograms/cubic meter (LC),Y
1,18.0,4.4,None,None,Quincy,Plumas,063,2019-03-30,2019-08-05,NAD83,...,88101,1,PM25 24-hour 2006,24 HOUR,"267 N. CHURCH ST., QUINCY, CA. 95971",1006,California,06,Micrograms/cubic meter (LC),Y
2,18.0,4.4,None,None,Quincy,Plumas,063,2019-03-29,2019-08-05,NAD83,...,88101,1,PM25 24-hour 2006,24 HOUR,"267 N. CHURCH ST., QUINCY, CA. 95971",1006,California,06,Micrograms/cubic meter (LC),Y
3,20.0,4.8,None,None,Quincy,Plumas,063,2019-03-28,2019-08-05,NAD83,...,88101,1,PM25 24-hour 2006,24 HOUR,"267 N. CHURCH ST., QUINCY, CA. 95971",1006,California,06,Micrograms/cubic meter (LC),Y
4,27.0,6.5,None,None,Quincy,Plumas,063,2019-03-27,2019-08-05,NAD83,...,88101,1,PM25 24-hour 2006,24 HOUR,"267 N. CHURCH ST., QUINCY, CA. 95971",1006,California,06,Micrograms/cubic meter (LC),Y


In [96]:
total_county_data_40.to_csv("ca_40.csv")

In [74]:
county_codes = city_df['code'].tolist()
county_codes_30 = county_codes[21:30]
county_codes_30

['043', '045', '047', '049', '051', '053', '055', '057', '059']

In [94]:
total_county_data_30 = pd.DataFrame()
for i in county_codes_30:
    county_data = "https://aqs.epa.gov/data/api/dailyData/byCounty?email="+gkey+"&key="+key+"&param=88101&bdate=20140301&edate=20190331&state=06&county="+i
    print(county_data)
    response = requests.get(county_data)
    response_json = response.json()
    response_json_data = response_json['Data']
    county_data =pd.DataFrame.from_dict(response_json_data)
    total_county_data_30 = total_county_data_30.append(county_data)
    time.sleep(2)
total_county_data_30.head()

https://aqs.epa.gov/data/api/dailyData/byCounty?email=janecc.gan@gmail.com&key=greywolf16&param=88101&bdate=20140301&edate=20190331&state=06&county=043
https://aqs.epa.gov/data/api/dailyData/byCounty?email=janecc.gan@gmail.com&key=greywolf16&param=88101&bdate=20140301&edate=20190331&state=06&county=045
https://aqs.epa.gov/data/api/dailyData/byCounty?email=janecc.gan@gmail.com&key=greywolf16&param=88101&bdate=20140301&edate=20190331&state=06&county=047
https://aqs.epa.gov/data/api/dailyData/byCounty?email=janecc.gan@gmail.com&key=greywolf16&param=88101&bdate=20140301&edate=20190331&state=06&county=049
https://aqs.epa.gov/data/api/dailyData/byCounty?email=janecc.gan@gmail.com&key=greywolf16&param=88101&bdate=20140301&edate=20190331&state=06&county=051
https://aqs.epa.gov/data/api/dailyData/byCounty?email=janecc.gan@gmail.com&key=greywolf16&param=88101&bdate=20140301&edate=20190331&state=06&county=053
https://aqs.epa.gov/data/api/dailyData/byCounty?email=janecc.gan@gmail.com&key=greywolf1

,aqi,arithmetic_mean,cbsa,cbsa_code,city,county,county_code,date_local,date_of_last_change,datum,...,parameter_code,poc,pollutant_standard,sample_duration,site_address,site_number,state,state_code,units_of_measure,validity_indicator
0,18.0,4.2,"Ukiah, CA",46380,Ukiah,Mendocino,045,2019-03-31,2019-07-01,WGS84,...,88101,3,PM25 24-hour 2006,24-HR BLK AVG,"105 N MAIN ST, UKIAH, CA 95482",0006,California,06,Micrograms/cubic meter (LC),Y
1,18.0,4.3,"Ukiah, CA",46380,Ukiah,Mendocino,045,2019-03-30,2019-07-01,WGS84,...,88101,3,PM25 24-hour 2006,24-HR BLK AVG,"105 N MAIN ST, UKIAH, CA 95482",0006,California,06,Micrograms/cubic meter (LC),Y
2,24.0,5.7,"Ukiah, CA",46380,Ukiah,Mendocino,045,2019-03-29,2019-07-01,WGS84,...,88101,3,PM25 24-hour 2006,24-HR BLK AVG,"105 N MAIN ST, UKIAH, CA 95482",0006,California,06,Micrograms/cubic meter (LC),Y
3,26.0,6.3,"Ukiah, CA",46380,Ukiah,Mendocino,045,2019-03-28,2019-07-01,WGS84,...,88101,3,PM25 24-hour 2006,24-HR BLK AVG,"105 N MAIN ST, UKIAH, CA 95482",0006,California,06,Micrograms/cubic meter (LC),Y
4,17.0,4.0,"Ukiah, CA",46380,Ukiah,Mendocino,045,2019-03-27,2019-07-01,WGS84,...,88101,3,PM25 24-hour 2006,24-HR BLK AVG,"105 N MAIN ST, UKIAH, CA 95482",0006,California,06,Micrograms/cubic meter (LC),Y


In [95]:
total_county_data_30.to_csv("ca_30.csv")

In [77]:
county_codes = city_df['code'].tolist()
county_codes_50 = county_codes[41:50]
county_codes_50

['083', '085', '087', '089', '091', '093', '095', '097', '099']

In [97]:
total_county_data_50 = pd.DataFrame()
for i in county_codes_50:
    county_data = "https://aqs.epa.gov/data/api/dailyData/byCounty?email="+gkey+"&key="+key+"&param=88101&bdate=20140301&edate=20190331&state=06&county="+i
    print(county_data)
    response = requests.get(county_data)
    response_json = response.json()
    response_json_data = response_json['Data']
    county_data =pd.DataFrame.from_dict(response_json_data)
    total_county_data_50 = total_county_data_50.append(county_data)
    time.sleep(2)
total_county_data_50.head()

https://aqs.epa.gov/data/api/dailyData/byCounty?email=janecc.gan@gmail.com&key=greywolf16&param=88101&bdate=20140301&edate=20190331&state=06&county=083
https://aqs.epa.gov/data/api/dailyData/byCounty?email=janecc.gan@gmail.com&key=greywolf16&param=88101&bdate=20140301&edate=20190331&state=06&county=085
https://aqs.epa.gov/data/api/dailyData/byCounty?email=janecc.gan@gmail.com&key=greywolf16&param=88101&bdate=20140301&edate=20190331&state=06&county=087
https://aqs.epa.gov/data/api/dailyData/byCounty?email=janecc.gan@gmail.com&key=greywolf16&param=88101&bdate=20140301&edate=20190331&state=06&county=089
https://aqs.epa.gov/data/api/dailyData/byCounty?email=janecc.gan@gmail.com&key=greywolf16&param=88101&bdate=20140301&edate=20190331&state=06&county=091
https://aqs.epa.gov/data/api/dailyData/byCounty?email=janecc.gan@gmail.com&key=greywolf16&param=88101&bdate=20140301&edate=20190331&state=06&county=093
https://aqs.epa.gov/data/api/dailyData/byCounty?email=janecc.gan@gmail.com&key=greywolf1

,aqi,arithmetic_mean,cbsa,cbsa_code,city,county,county_code,date_local,date_of_last_change,datum,...,parameter_code,poc,pollutant_standard,sample_duration,site_address,site_number,state,state_code,units_of_measure,validity_indicator
0,23.0,5.6,"Santa Maria-Santa Barbara, CA",42200,Santa Barbara,Santa Barbara,083,2019-03-31,2019-08-23,WGS84,...,88101,3,PM25 24-hour 2006,24-HR BLK AVG,700 E. CANON PERDIDO,0011,California,06,Micrograms/cubic meter (LC),Y
1,22.0,5.3,"Santa Maria-Santa Barbara, CA",42200,Santa Barbara,Santa Barbara,083,2019-03-30,2019-08-23,WGS84,...,88101,3,PM25 24-hour 2006,24-HR BLK AVG,700 E. CANON PERDIDO,0011,California,06,Micrograms/cubic meter (LC),Y
2,23.0,5.5,"Santa Maria-Santa Barbara, CA",42200,Santa Barbara,Santa Barbara,083,2019-03-29,2019-08-23,WGS84,...,88101,3,PM25 24-hour 2006,24-HR BLK AVG,700 E. CANON PERDIDO,0011,California,06,Micrograms/cubic meter (LC),Y
3,12.0,2.8,"Santa Maria-Santa Barbara, CA",42200,Santa Barbara,Santa Barbara,083,2019-03-28,2019-08-23,WGS84,...,88101,3,PM25 24-hour 2006,24-HR BLK AVG,700 E. CANON PERDIDO,0011,California,06,Micrograms/cubic meter (LC),Y
4,19.0,4.6,"Santa Maria-Santa Barbara, CA",42200,Santa Barbara,Santa Barbara,083,2019-03-27,2019-08-23,WGS84,...,88101,3,PM25 24-hour 2006,24-HR BLK AVG,700 E. CANON PERDIDO,0011,California,06,Micrograms/cubic meter (LC),Y


In [98]:
total_county_data_50.to_csv("ca_50.csv")

In [81]:
county_codes = city_df['code'].tolist()
county_codes_60 = county_codes[51:58]
county_codes_60

['103', '105', '107', '109', '111', '113', '115']

In [99]:
total_county_data_60 = pd.DataFrame()
for i in county_codes_60:
    county_data = "https://aqs.epa.gov/data/api/dailyData/byCounty?email="+gkey+"&key="+key+"&param=88101&bdate=20140301&edate=20190331&state=06&county="+i
    print(county_data)
    response = requests.get(county_data)
    response_json = response.json()
    response_json_data = response_json['Data']
    county_data =pd.DataFrame.from_dict(response_json_data)
    total_county_data_60 = total_county_data_60.append(county_data)
    time.sleep(2)
total_county_data_60.head()

https://aqs.epa.gov/data/api/dailyData/byCounty?email=janecc.gan@gmail.com&key=greywolf16&param=88101&bdate=20140301&edate=20190331&state=06&county=103
https://aqs.epa.gov/data/api/dailyData/byCounty?email=janecc.gan@gmail.com&key=greywolf16&param=88101&bdate=20140301&edate=20190331&state=06&county=105
https://aqs.epa.gov/data/api/dailyData/byCounty?email=janecc.gan@gmail.com&key=greywolf16&param=88101&bdate=20140301&edate=20190331&state=06&county=107
https://aqs.epa.gov/data/api/dailyData/byCounty?email=janecc.gan@gmail.com&key=greywolf16&param=88101&bdate=20140301&edate=20190331&state=06&county=109
https://aqs.epa.gov/data/api/dailyData/byCounty?email=janecc.gan@gmail.com&key=greywolf16&param=88101&bdate=20140301&edate=20190331&state=06&county=111
https://aqs.epa.gov/data/api/dailyData/byCounty?email=janecc.gan@gmail.com&key=greywolf16&param=88101&bdate=20140301&edate=20190331&state=06&county=113
https://aqs.epa.gov/data/api/dailyData/byCounty?email=janecc.gan@gmail.com&key=greywolf1

,aqi,arithmetic_mean,cbsa,cbsa_code,city,county,county_code,date_local,date_of_last_change,datum,...,parameter_code,poc,pollutant_standard,sample_duration,site_address,site_number,state,state_code,units_of_measure,validity_indicator
0,0.0,0.0,"Red Bluff, CA",39780,Red Bluff,Tehama,103,2019-03-31,2019-08-29,WGS84,...,88101,3,PM25 24-hour 2006,24-HR BLK AVG,"1834 Walnut Street, Red Bluff, CA 96080",0007,California,06,Micrograms/cubic meter (LC),Y
1,0.0,0.0,"Red Bluff, CA",39780,Red Bluff,Tehama,103,2019-03-30,2019-08-29,WGS84,...,88101,3,PM25 24-hour 2006,24-HR BLK AVG,"1834 Walnut Street, Red Bluff, CA 96080",0007,California,06,Micrograms/cubic meter (LC),Y
2,0.0,0.0,"Red Bluff, CA",39780,Red Bluff,Tehama,103,2019-03-29,2019-08-29,WGS84,...,88101,3,PM25 24-hour 2006,24-HR BLK AVG,"1834 Walnut Street, Red Bluff, CA 96080",0007,California,06,Micrograms/cubic meter (LC),Y
3,0.0,0.0,"Red Bluff, CA",39780,Red Bluff,Tehama,103,2019-03-28,2019-08-29,WGS84,...,88101,3,PM25 24-hour 2006,24-HR BLK AVG,"1834 Walnut Street, Red Bluff, CA 96080",0007,California,06,Micrograms/cubic meter (LC),Y
4,0.0,0.0,"Red Bluff, CA",39780,Red Bluff,Tehama,103,2019-03-27,2019-08-29,WGS84,...,88101,3,PM25 24-hour 2006,24-HR BLK AVG,"1834 Walnut Street, Red Bluff, CA 96080",0007,California,06,Micrograms/cubic meter (LC),Y


In [100]:
total_county_data_60.to_csv("ca_60.csv")

In [88]:
county_codes = city_df['code'].tolist()
county_codes_20 = county_codes[11:20]
county_codes_20

['023', '025', '027', '029', '031', '033', '035', '037', '039']

In [101]:
total_county_data_20 = pd.DataFrame()
for i in county_codes_20:
    county_data = "https://aqs.epa.gov/data/api/dailyData/byCounty?email="+gkey+"&key="+key+"&param=88101&bdate=20140301&edate=20190331&state=06&county="+i
    print(county_data)
    response = requests.get(county_data)
    response_json = response.json()
    response_json_data = response_json['Data']
    county_data =pd.DataFrame.from_dict(response_json_data)
    total_county_data_20 = total_county_data_20.append(county_data)
    time.sleep(2)
total_county_data_20.head()

https://aqs.epa.gov/data/api/dailyData/byCounty?email=janecc.gan@gmail.com&key=greywolf16&param=88101&bdate=20140301&edate=20190331&state=06&county=023
https://aqs.epa.gov/data/api/dailyData/byCounty?email=janecc.gan@gmail.com&key=greywolf16&param=88101&bdate=20140301&edate=20190331&state=06&county=025
https://aqs.epa.gov/data/api/dailyData/byCounty?email=janecc.gan@gmail.com&key=greywolf16&param=88101&bdate=20140301&edate=20190331&state=06&county=027
https://aqs.epa.gov/data/api/dailyData/byCounty?email=janecc.gan@gmail.com&key=greywolf16&param=88101&bdate=20140301&edate=20190331&state=06&county=029
https://aqs.epa.gov/data/api/dailyData/byCounty?email=janecc.gan@gmail.com&key=greywolf16&param=88101&bdate=20140301&edate=20190331&state=06&county=031
https://aqs.epa.gov/data/api/dailyData/byCounty?email=janecc.gan@gmail.com&key=greywolf16&param=88101&bdate=20140301&edate=20190331&state=06&county=033
https://aqs.epa.gov/data/api/dailyData/byCounty?email=janecc.gan@gmail.com&key=greywolf1

,aqi,arithmetic_mean,cbsa,cbsa_code,city,county,county_code,date_local,date_of_last_change,datum,...,parameter_code,poc,pollutant_standard,sample_duration,site_address,site_number,state,state_code,units_of_measure,validity_indicator
0,26.0,6.2,"Eureka-Arcata-Fortuna, CA",21700,Not in a city,Humboldt,023,2015-04-19,2016-06-16,WGS84,...,88101,1,PM25 24-hour 2006,24-HR BLK AVG,"5 Loop Rd Hoopa, CA 55546",0006,California,06,Micrograms/cubic meter (LC),Y
1,28.0,6.6,"Eureka-Arcata-Fortuna, CA",21700,Not in a city,Humboldt,023,2015-04-18,2016-06-16,WGS84,...,88101,1,PM25 24-hour 2006,24-HR BLK AVG,"5 Loop Rd Hoopa, CA 55546",0006,California,06,Micrograms/cubic meter (LC),Y
2,29.0,7.0,"Eureka-Arcata-Fortuna, CA",21700,Not in a city,Humboldt,023,2015-04-17,2016-06-16,WGS84,...,88101,1,PM25 24-hour 2006,24-HR BLK AVG,"5 Loop Rd Hoopa, CA 55546",0006,California,06,Micrograms/cubic meter (LC),Y
3,23.0,5.6,"Eureka-Arcata-Fortuna, CA",21700,Not in a city,Humboldt,023,2015-04-16,2016-06-16,WGS84,...,88101,1,PM25 24-hour 2006,24-HR BLK AVG,"5 Loop Rd Hoopa, CA 55546",0006,California,06,Micrograms/cubic meter (LC),Y
4,15.0,3.6,"Eureka-Arcata-Fortuna, CA",21700,Not in a city,Humboldt,023,2015-04-15,2016-06-16,WGS84,...,88101,1,PM25 24-hour 2006,24-HR BLK AVG,"5 Loop Rd Hoopa, CA 55546",0006,California,06,Micrograms/cubic meter (LC),Y


In [102]:
total_county_data_20.to_csv("ca_20.csv")

In [103]:
frames = [total_county_data_10, total_county_data_20, total_county_data_30,total_county_data_40,total_county_data_50,total_county_data_60]

In [111]:
combined_ca = pd.concat(frames)
combined_ca.head()

,aqi,arithmetic_mean,cbsa,cbsa_code,city,county,county_code,date_local,date_of_last_change,datum,...,parameter_code,poc,pollutant_standard,sample_duration,site_address,site_number,state,state_code,units_of_measure,validity_indicator
0,18.0,4.2,"San Francisco-Oakland-Hayward, CA",41860,Livermore,Alameda,001,2019-03-31,2019-08-27,WGS84,...,88101,3,PM25 24-hour 2006,24-HR BLK AVG,793 Rincon Ave.,0007,California,06,Micrograms/cubic meter (LC),Y
1,16.0,3.9,"San Francisco-Oakland-Hayward, CA",41860,Livermore,Alameda,001,2019-03-30,2019-08-27,WGS84,...,88101,3,PM25 24-hour 2006,24-HR BLK AVG,793 Rincon Ave.,0007,California,06,Micrograms/cubic meter (LC),Y
2,8.0,1.8,"San Francisco-Oakland-Hayward, CA",41860,Livermore,Alameda,001,2019-03-29,2019-08-27,WGS84,...,88101,3,PM25 24-hour 2006,24-HR BLK AVG,793 Rincon Ave.,0007,California,06,Micrograms/cubic meter (LC),Y
3,19.0,4.5,"San Francisco-Oakland-Hayward, CA",41860,Livermore,Alameda,001,2019-03-28,2019-08-27,WGS84,...,88101,3,PM25 24-hour 2006,24-HR BLK AVG,793 Rincon Ave.,0007,California,06,Micrograms/cubic meter (LC),Y
4,9.0,2.1,"San Francisco-Oakland-Hayward, CA",41860,Livermore,Alameda,001,2019-03-27,2019-08-27,WGS84,...,88101,3,PM25 24-hour 2006,24-HR BLK AVG,793 Rincon Ave.,0007,California,06,Micrograms/cubic meter (LC),Y


In [112]:
combined_ca.count()

aqi                    651335
arithmetic_mean        780214
cbsa                   747466
cbsa_code              747466
city                   780214
county                 780214
county_code            780214
date_local             780214
date_of_last_change    780214
datum                  780214
event_type             780214
first_max_hour         780214
first_max_value        780214
latitude               780214
local_site_name        779904
longitude              780214
method                 780214
method_code            780214
observation_count      780214
observation_percent    780214
parameter              780214
parameter_code         780214
poc                    780214
pollutant_standard     651335
sample_duration        780214
site_address           780214
site_number            780214
state                  780214
state_code             780214
units_of_measure       780214
validity_indicator     780214
dtype: int64

In [129]:
ca_clean = combined_ca[['aqi','cbsa','city','county','county_code','date_local','parameter_code','sample_duration','pollutant_standard','latitude','longitude','state','state_code']]
ca_clean.head()

,aqi,cbsa,city,county,county_code,date_local,parameter_code,sample_duration,pollutant_standard,latitude,longitude,state,state_code
0,18.0,"San Francisco-Oakland-Hayward, CA",Livermore,Alameda,001,2019-03-31,88101,24-HR BLK AVG,PM25 24-hour 2006,37.687526,-121.784217,California,06
1,16.0,"San Francisco-Oakland-Hayward, CA",Livermore,Alameda,001,2019-03-30,88101,24-HR BLK AVG,PM25 24-hour 2006,37.687526,-121.784217,California,06
2,8.0,"San Francisco-Oakland-Hayward, CA",Livermore,Alameda,001,2019-03-29,88101,24-HR BLK AVG,PM25 24-hour 2006,37.687526,-121.784217,California,06
3,19.0,"San Francisco-Oakland-Hayward, CA",Livermore,Alameda,001,2019-03-28,88101,24-HR BLK AVG,PM25 24-hour 2006,37.687526,-121.784217,California,06
4,9.0,"San Francisco-Oakland-Hayward, CA",Livermore,Alameda,001,2019-03-27,88101,24-HR BLK AVG,PM25 24-hour 2006,37.687526,-121.784217,California,06


In [131]:
ca_clean = ca_clean.loc[ca_clean['pollutant_standard'] == "PM25 24-hour 2012"]
ca_clean =ca_clean.loc[ca_clean['sample_duration'] != "1 HOUR"]
ca_clean.count()

aqi                   162822
cbsa                  156150
city                  162822
county                162822
county_code           162822
date_local            162822
parameter_code        162822
sample_duration       162822
pollutant_standard    162822
latitude              162822
longitude             162822
state                 162822
state_code            162822
dtype: int64

In [133]:
ca_clean.to_excel("ca_clean.xlsx")


In [12]:
#choose only "pm2.5 24h standard"
sf_df_new = sf_df.loc[sf_df['pollutant_standard']=="PM25 24-hour 2012"]
sf_df_new.head()

,aqi,arithmetic_mean,cbsa,cbsa_code,city,county,county_code,date_local,date_of_last_change,datum,...,parameter_code,poc,pollutant_standard,sample_duration,site_address,site_number,state,state_code,units_of_measure,validity_indicator
5444,28.0,6.7,"San Francisco-Oakland-Hayward, CA",41860,San Francisco,San Francisco,075,2019-03-31,2019-08-27,WGS84,...,88101,3,PM25 24-hour 2012,24-HR BLK AVG,10 ARKANSAS ST.,0005,California,06,Micrograms/cubic meter (LC),Y
5445,22.0,5.2,"San Francisco-Oakland-Hayward, CA",41860,San Francisco,San Francisco,075,2019-03-30,2019-08-27,WGS84,...,88101,3,PM25 24-hour 2012,24-HR BLK AVG,10 ARKANSAS ST.,0005,California,06,Micrograms/cubic meter (LC),Y
5446,23.0,5.4,"San Francisco-Oakland-Hayward, CA",41860,San Francisco,San Francisco,075,2019-03-29,2019-08-27,WGS84,...,88101,3,PM25 24-hour 2012,24-HR BLK AVG,10 ARKANSAS ST.,0005,California,06,Micrograms/cubic meter (LC),Y
5447,24.0,5.7,"San Francisco-Oakland-Hayward, CA",41860,San Francisco,San Francisco,075,2019-03-28,2019-08-27,WGS84,...,88101,3,PM25 24-hour 2012,24-HR BLK AVG,10 ARKANSAS ST.,0005,California,06,Micrograms/cubic meter (LC),Y
5448,15.0,3.5,"San Francisco-Oakland-Hayward, CA",41860,San Francisco,San Francisco,075,2019-03-27,2019-08-27,WGS84,...,88101,3,PM25 24-hour 2012,24-HR BLK AVG,10 ARKANSAS ST.,0005,California,06,Micrograms/cubic meter (LC),Y


In [13]:
#get min,max,mean
aqi_min = sf_df_new['aqi'].min()
aqi_max =sf_df_new['aqi'].max()
aqi_mean = sf_df_new['aqi'].mean()

print(aqi_min)
print(aqi_max)
print(aqi_mean)


0.0
228.0
33.141828254847645


In [14]:
# summurize data in categories
size_bins = [0,50,100,150,200,300,500]
size_names= ['good','Moderate','Unhealthy for sensitive groups','unhealthy','very unhealthy','hazardous']
sf_df_bin = sf_df_new[['date_local']].groupby(pd.cut(sf_df_new['aqi'],bins=size_bins,labels=size_names)).count()
sf_df_bin

,date_local
aqi,
good,1440
Moderate,296
Unhealthy for sensitive groups,9
unhealthy,11
very unhealthy,1
hazardous,0


## Step 2: export data to the database

In [132]:
#activate mongod environment in terminal before running this cell
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

# 'testdb' is the name of the database
db = client.ca_data

# testcollection is the name of the collection
collection = db['ca_data']

# this line stays the same
collection.insert_many(ca_clean.to_dict('records'))
